# train, test 불러오기 - xlsx (엑셀)

In [38]:
import xlrd

workbook = xlrd.open_workbook("train_test.xlsx")

# train
worksheet = workbook.sheet_by_index(0) # train sheet에 접근합니다.
train = worksheet.col_values(3) # document_morpheme 열(3 번째 열)의 값들을 document_morpheme 변수에 저장합니다.
y_train = worksheet.col_values(2)

# test
worksheet = workbook.sheet_by_index(1) # test sheet에 접근합니다.
test = worksheet.col_values(3) # document_morpheme 열(3 번째 열)의 값들을 document_morpheme 변수에 저장합니다.
y_test = worksheet.col_values(2)

In [39]:
type(train)

list

In [40]:
train

['아 아 더빙 진짜 짜증나 네요 목소리',
 '흠 포스터 보고 초 딩 영화 줄 오버 연기 조차 가볍 지 않 구나',
 '너무 재 밓 어 었 다 그래서 보 는 것 을 추천 하 ㄴ다',
 '교도소 이야기 구 멀 ㄴ 솔직히 재미 는 없 다 평점 조정',
 '사이몬페그 의 익살 스럽 ㄴ 연기 가 돋보이 었 더 ㄴ 영화 스파이 더 맨 에서 늙 어 보이 기 만 하 었 더 ㄴ 커스틴 던스트 가 너무나 도 이쁘 어 보이 었 다',
 '막 걸음마 떼 ㄴ 세 부터 초등학교 학년생 이 ㄴ 살 용 영화 ㅋㅋㅋ 별 반개 도 아깝 ㅁ',
 '원작 의 긴장감 을 제대로 살리 어 내 지 못하 었 다',
 '별 반개 도 아깝 다 욕 나오 ㄴ다 이응 경 길 용 우 연기 생활 이 몇 년 이 ㄴ지 정말 발 로 하 어도 그것 보다 ㄴ 낫 겠 다 납치 감금 만 반복 반복 이 드라마 는 가족 도 없 다 연기 못 하 는 사람 만 모 엿 네',
 '액션 이 없 는데 도 재미 있 는 몇 안 되 는 영화',
 '왜케 평점 이 낮 은 것 이 ㄴ데 꽤 보 ㄹ 만하 ㄴ데 허 ㄹ 리 우드 식 화려 하 ㅁ 에 만 너무 길들이 어 지 어 있 나',
 '걍 인피 니트 가 짱 이 다 진짜 짱 이 다',
 '보 ㄹ 때 마다 눈물 나 아서 죽 겠 다 녀 ㄴ 대의 향수 자극 허 진호 는 감성 절제 멜로 의 달인 이 다',
 '울 면서 손들 고 횡단보도 건너 ㄹ 때 뛰쳐나오 ㄹ 뻔 이 범수 연기 드럽 게 못하 어',
 '담백 하 고 깔끔 하 어서 좋 다 신문 기사 로 만 보다 보 면 자꾸 잊어버리 ㄴ다 그 들 도 사람 이 었 다는 것 을',
 '취향 은 존중 하 ㄴ다지만 진짜 내생 에 극장 에서 보 ㄴ 영화 중 가장 노 잼 노 감동 임 스토리 도 어거지 이 고 감동 도 어거지',
 'ㄱ 냥 매번 긴장 되 고 재 밋 음 ㅠㅠ',
 '참 사람 들 웃기 ㄴ 것 이 바스코 가 이기 면 락스코 라고 까 고 바 비 가 이기 면 아이 돌이 라고 까 ㄴ다 그냥 까 고 싶 어서 안달 낳 ㄴ 것 처럼 보이 ㄴ다',
 '굿바이 레 니 ㄴ 표절 이

# 정수 인코딩

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(train)

# 벡터화
x_train = tokenizer.texts_to_sequences(train)
x_test = tokenizer.texts_to_sequences(test)

In [42]:
# 총 단어 수
total_word = len(tokenizer.word_index)

# 단어 집합의 크기
vocab_size = total_word+1

#
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

In [5]:
import pickle

# 만들어진 tokenizer를 저장해줍니다. - 추 후에 받을 데이터를 정수 인코딩 해주기 위함.
with open('tokenizer.p', 'wb') as file: # 파일을 바이너리 쓰기 모드(wb)로 열기 
    pickle.dump(tokenizer, file) # 저장하기

In [6]:
import pandas as pd

# 데이터의 최대 길이 확인

# train + test 데이터 합치기
all_data = train + test

max_len = 0
for i in all_data:
    if max_len <= len(i):
        max_len = len(i)
        
print(max_len)

264


In [7]:
# 패딩을 통해 길이를 맞춰줍니다.
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen = 100)
x_test = pad_sequences(x_test, maxlen = 80)

In [8]:
x_test.shape

(49572, 80)

In [9]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape, max_len, vocab_size

((148738, 100), (49572, 80), (148738,), (49572,), 264, 39682)

# 모델1 - 기본 dense layer

In [10]:
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.models import Sequential

### 모델 구축

In [11]:
model__ = Sequential()

model__.add(Embedding(vocab_size, 128)) # 21일 주어진 데이터를 예측하기 위해, embedding 층을 처음에 넣어줘야 합니다.
model__.add(LSTM(64, activation='relu'))
model__.add(Dropout(0.5))
model__.add(Dense(32, activation='relu'))
model__.add(Dense(1, activation='sigmoid')) # output shape을 (None, 1)로 맞춰줍니다. - 추 후, 주어진 데이터을 이용하여 모델의 예측한 값을 뽑기위해...

model__.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model__.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         5079296   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,130,817
Trainable params: 5,130,817
Non-trainable params: 0
_________________________________________________________________


### 모델 학습

In [13]:
model__.fit(x_train, y_train, epochs = 1, batch_size = 64, validation_split = 0.2)

1860/1860 [==============================] - 239s 129ms/step - loss: 5271090176.0000 - accuracy: 0.8049 - val_loss: 0.3608 - val_accuracy: 0.8425


In [37]:
# loaded_model = load_model('best_model.h5')
print(model.evaluate(x_test, y_test))

1550/1550 [==============================] - 25s 16ms/step - loss: 0.3998 - accuracy: 0.8399
[0.3998051583766937, 0.8398894667625427]


### 모델 저장 // 나중에 다시 load 해서 test 데이터로 바로 예측하기 위함.

In [14]:
model__.save("model.h5")

# 모델 로드 후, 주어진 데이터로 예측하기

### 모델 로드

In [15]:
# from keras.models import load_model
from tensorflow.keras.models import load_model

In [16]:
model = load_model("model.h5")

### 주어진 데이터 로드 및 정수 인코딩

In [19]:
# 데이터 로드

import xlrd

# 해당 파일은 네이버 영화 리뷰 데이터입니다.
workbook = xlrd.open_workbook("test.xlsx") 

# test
worksheet = workbook.sheet_by_index(0) # test sheet에 접근합니다.
test = worksheet.col_values(1) # document 열에 접근

In [ ]:
#import xlrd

#workbook = xlrd.open_workbook("test.xlsx")

# train
#worksheet = workbook.sheet_by_index(0) # test sheet에 접근합니다.
#test = worksheet.col_values(0) # 기본 data
#test_morpheme = worksheet.col_values(1) # kkma를 이용하여 형태소 분석한 결과
#test_syllable = worksheet.col_values(2) # 음절별로 나눈 결과

In [22]:
test

['노래 가 너무 적 음',
 '돌 겠 네 진짜 황 숙아 어크 공장 그만 돌리 어 라 죽 는 다',
 '막노동 체험 판 막노동 하 는 사람 이 ㄴ데 장비 를 내가 사 아야 되 어 무엇 이 지',
 '차 악차 악차 악 정말 이래 서 왕국 을 되찾 을 수 있 는 거 야',
 '시간 때우 기에 좋 음 도전 과제 는 시간 이 면 다 깨 ㄹ 수 있 어요',
 '역시 재미있 네요 전 작 에서 하 ㄹ 수 없 었 더 ㄴ 자유 롭 ㄴ 덱 빌딩 도 좋 네요',
 '재미있 었 습니다',
 '은근 쉽 지만 은근 어렵 ㄴ 게임',
 '베 ㅈ 스 다 이 개 ㅐ 끼들 아 아 시작 하 ㄹ 때 체스 판 돌아가 는 거 분 동안 번 보 았 더 나 ㅈ ㄴ 빡 치 네 진짜 무 한로 딩 버그 안 치 어 고치 냐 겜 하 지 말르 아 는 걸 냐',
 '시간 분 동안 스트레스 다 풀리 ㅁ ㄹ ㅇ',
 '걍 겜 임 계정 을 못 만들 어 미치 ㄴ 겜',
 '관람객 호랑이 불가',
 '재미 쩡 초반 에 좀 힘들 어 여 도트 라서 조아',
 '포켓볼 도 몰르 았 는데 이 것 으로 배우 어 갈 ㅂ니다 심심 하 ㄹ 때 하면 좋 아요 컴퓨터 상대 하 는 거 제대 로 이기 어 보 고 싶 은데 잘 안되 네요',
 '트레이 아크 사랑 하 어요 핰 핰',
 '좆 망 겜 하 지 마 무슨 처음 에 키 자마자 매칭 이 왜 렙 하고 매칭 되 냐',
 '뭐 랄 까 아 다 크 소 우리 하 는 느낌 이 에요 죽 으면 돈 다 날라가 는 방식 렙 업 하 면서 무한 반복 이 ㄴ 거 같 은데 재미있 습니다 클래스 도 은근 많 은 거 같 고 또 캐릭터 를 계속 추가 하 ㄹ 예정 이 ㄴ가 같 고 그리 고 하 ㄹ 요소 하 고 매번 덜 ㄴ 전도 바뀌 고 재미있 습니다 돈 이 아깝 지 않 다고 생각 들 네요 ㅊ ㅊ',
 '원하 는 거 보 실리 어 면 홈페이지 가시 어서 패치 를 사 시 어야 하 ㅂ니다 패치 가 만원 이 라 사실상 만원 주고 게임 을 사 는 것 이 ㄴ데 ㅁ 좋 던데요',
 '안해 썩 을 까 아 진 그냥 대충 슥슥해 도 되 더니 에 서 판

In [20]:
# tokenzier load

with open('tokenizer.p', 'rb') as file: # 파일을 바이너리 읽기 모드(rb)로 열기
    tokenizer = pickle.load(file)

In [21]:
test = list(map(str, test))

# 벡터화
test_data = tokenizer.texts_to_sequences(test)

In [23]:
max_len = 0
for i in test_data:
    if max_len <= len(i):
        max_len = len(i)
        
# 패딩을 통해 길이를 맞춰줍니다.
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_data = pad_sequences(test_data, maxlen = max_len)

### 모델 예측 // 21일, 주어진 데이터로 만들어놓은 모델을 사용해 예측해 줍니다.

In [24]:
y_pred = model__.predict_classes(test_data).reshape(-1)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [25]:
print(y_pred)

[1 0 0 ... 0 1 0]


In [32]:
y_pred

array([1, 0, 0, ..., 0, 1, 0])

### 예측한 값을 엑셀 파일로 저장

In [26]:
import openpyxl

In [27]:
from openpyxl import Workbook

write = Workbook()
write_ = write.create_sheet("y값")

write_ = write.active

# 값 추가 / 행 단위
for i in y_pred:
    write_.append([i])
    
# 저장
write.save("예측한 값.xlsx")